In [24]:
from PIL import Image
import numpy as np
from copy import deepcopy

Вы работаете в ФСБ в отделе К подотделе шифрования.
Спецагенты схватили трех шпионов и у каждого из них были изъяты улики.
1. у певрого была бумашка со следующим содержимым: 1f933b5
2. у второго была фотография 11.png
3. у третьего была фотография 222.bmp

После некоторых рассуждений вы решили начать с числа и для начала перевести его в десятичное представление

In [ ]:
def hex_to_dec(value: str) -> str:
    """
    Функция преобразует строку содержащую число в шестнадцатеричной системе счисления
    в строку содержащую число в десятичной системе счисления
    """
    pass

In [ ]:
v1 = hex_to_dec("1f933b5")

In [ ]:
a   b  c  d  e  f
10 11 12 13 14 15

In [88]:
5 + 11*16 + 3*16**2 + 3 * 16**3 + 9*16**4 + 15 * 16**5 + 1 * 16 **6

33108917

In [91]:
v1 = 5 + 11*16 + 3*16**2 + 3 * 16**3 + 9*16**4 + 15 * 16**5 + 1 * 16 **6

После получения числа в десятичном виде и осознания что другие данные связаны с изображениями. Вы решили попытаться понять какие еще числа вы можете получить из данного и решили разложить его на простые

In [90]:
def devisors(value: int):
    """
    Фукнция раскладывает число на простые множители
    На вход подается только числа у которых таких множителей либо два, либо один (то есть оно уже простое).
    
    value: Натуральное число
    
    return Список двух простых чисел произведение которых дает исходное число
    
    """
    for i in range(2, int(np.sqrt(value))):
        if value // i == value / i:
            break
    return value//i, value//(value//i)

In [93]:
v2,v3 = devisors(v1)
v2,v3

(7369, 4493)

Получив два числа вы увидели странности на изображении 11.png в верхнем левом углу и решили что там шифровка. Для начала вы попробовали преобразовать цифры в двоичный код и наложить на огрехи в изображении

In [96]:
def dec_to_bin(value: int) -> str:
    """
    Функция преобразует строку содержащую число из десятичной системы счисления 
    в строку содержащую число в двоичной системе счисления
    """
    return bin(value)[2:]

In [97]:
v4 = dec_to_bin(v2)
v5 = dec_to_bin(v3)

v4, v5

('1110011001001', '1000110001101')

In [100]:
len(v4), len(v5)

(13, 13)

In [ ]:
После чего вы выбрали те байты из изображения что находились на перекрестии строк и столбцов 
которые отвечали единицам двоичных чисел если их прислонить к изображению в верхний левый угол меньшее число
по вертикали а большее по горизонтали

In [99]:
filepath = "11.png"
with Image.open(filepath) as img:
    img.load()
    img = np.asarray(img)
new_img = deepcopy(img)

  101
 
1 101
0 000
1 101

In [101]:
new_img[:13, :13]

array([[ 83,  75, 102,   0,   0, 102,  84,   0,   0,  94,   0,   0,  78],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 32, 103,  75,   0,   0,  88,  87,   0,   0, 101,   0,   0, 102],
       [102,  88,  84,   0,   0,  88, 102,   0,   0, 102,   0,   0,  80],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 88,  84, 102,   0,   0,  83,  78,   0,   0,  93,   0,   0,  75],
       [ 73,  84, 102,   0,   0, 102,  83,   0,   0,  75,   0,   0, 102],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [102,  74,  75,   0,   0,  81, 103,   0,   0,  75,   0,   0,  88]],
      dtype=uint8)

In [103]:
def get_bytes(bin1, bin2, img):
    res = []
    for rowi in range(len(v5)):
        for coli in range(len(v4)):
            if v5[rowi] == "1" and v4[coli] == "1":
                res.append(img[rowi, coli])
                
    return res

In [104]:
byte_array = get_bytes(v5, v4, new_img)
byte_array

[83,
 75,
 102,
 102,
 84,
 94,
 78,
 32,
 103,
 75,
 88,
 87,
 101,
 102,
 102,
 88,
 84,
 88,
 102,
 102,
 80,
 88,
 84,
 102,
 83,
 78,
 93,
 75,
 73,
 84,
 102,
 102,
 83,
 75,
 102,
 102,
 74,
 75,
 81,
 103,
 75,
 88]

Далее вы посмотрев на байты изображения уловили в них отголоски кодировки ascii и решили с помощью chr преобразовать их в символы, только всем кодам кроме 32 вы решили добавить 1000 чтобы быть в области русских букв

In [110]:
1 if 20 < 11 else 0

0

In [112]:
chr(1032)

'Ј'

In [114]:
def get_chars(byte_array):
    res = []
    for el in byte_array:
        if el == 32:
            res.append(chr(32))
        else:
            res.append(chr(el+1000))
    return res

In [ ]:
def get_chars(byte_array):
    res = []
    for el in byte_array:
        val = chr(32) if el == 32 else chr(el+1000)
        res.append(val)
    return res

In [107]:
def get_chars(byte_array):
    return list([chr(el + 1000) if el != 32 else " " for el in byte_array])

In [108]:
chars = get_chars(byte_array)
chars

['л',
 'г',
 'ю',
 'ю',
 'м',
 'ц',
 'ж',
 ' ',
 'я',
 'г',
 'р',
 'п',
 'э',
 'ю',
 'ю',
 'р',
 'м',
 'р',
 'ю',
 'ю',
 'и',
 'р',
 'м',
 'ю',
 'л',
 'ж',
 'х',
 'г',
 'б',
 'м',
 'ю',
 'ю',
 'л',
 'г',
 'ю',
 'ю',
 'в',
 'г',
 'й',
 'я',
 'г',
 'р']

Вы получили какую-то не очень длинную строчку. Возможно это ключ, но он тоже зашифрован. Вы решили попробовать самую простую кодировку - шифр Цезаря. И путем перебора расшифровали ключ. Как возможные символы попробовали взять алфавит с пробелом.

In [115]:
s = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "
len(s)

34

In [121]:
def caesar_encryptor(text:str, n:int) -> str:
    """
    Функция шифрует шифром Цезаря с шагом n (n это ключ) русские предложения (пробел добавляется в конец алфавита, 
    заглавные буквы идут перед строчными: "АБВ...Яабв...я ") знаки припинания игнорируются. По достижению конца
    алфавита отсчет начинается сначала.
    Пусть например n = 2
    Шифр цезаря это когда вместо А пишется В, вместо Б пишет Г сдвиг на две буквы
    
    caesar_encryptor("Всем привет!", 1) -> "ГтёнАрсйгёу"
    """
    pass


def caesar_decryptor(text:str, step) -> str:
    """
    Расшифровывает Шифр Цезаря 
    caesar_decryptor(caesar_encryptor("Всем привет!", 1), 1) -> "Всем привет"
    """
    s = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "
    res = []
    for ch in text:
        res.append(s[s.index(ch) - step])
    return "".join(res)
    

def caesar_brootforce(text:str):
    """
    Перебор смещений пока не получится вменяемая строка
    """
    s = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "
    for i in range(len(s)):
        print(i, caesar_decryptor(text, i))

In [122]:
caesar_brootforce("".join(chars))

0 лгююмцж ягрпэююрмрююирмюлжхгбмююлгюювгйягр
1 квээлхёяювпоьээплпээзплэкёфвалээквээбвиювп
2 йбьькфеюэбоныььокоььжокьйеуб кььйбььабзэбо
3 иаыыйудэьанмъыынйныыёнйыидтаяйыыиаыы ажьан
4 з ъъитгьы млщъъмимъъемиъзгс юиъъз ъъя ёы м
5 жящщзсвыъялкшщщлзлщщдлзщжвряэзщщжящщюяеъял
6 ёюшшжрбъщюкйчшшкжкшшгкжшёбпюьжшшёюшшэюдщюк
7 еэччёпащшэйицччйёйччвйёчеаоэыёччеэччьэгшэй
8 дьццео шчьизхцциеиццбиецд ньъеццдьццыьвчьи
9 гыххднячцызжфххздзххаздхгямыщдххгыххъыбцыз
10 въффгмюцхъжёуффжгжфф жгфвюлъшгффвъффщъахъж
11 бщуувлэхфщёетууёвёууяёвубэкщчвуубщуушщ фщё
12 ашттбкьфушедсттебеттюебтаьйшцбтташттчшяуше
13  чссайыутчдгрссдадссэдас ыичхасс чссцчютчд
14 яцрр иътсцгвпррг гррьг ряъзцф рряцррхцэсцг
15 юхппязщсрхвбоппвявппывяпющжхуяппюхппфхьрхв
16 эфооюжшрпфбанообюбооъбюоэшёфтюооэфооуфыпфб
17 ьуннэёчпоуа мннаэаннщаэньчеусэнньуннтуъоуа
18 ытммьецонт ялмм ь ммш ьмыцдтрьммытммстщнт 
19 ъсллыдхнмсяюклляыяллчяылъхгспыллъсллрсшмся
20 щрккъгфмлрюэйккюъюккцюъкщфвроъккщрккпрчлрю
21 шпййщвулкпэьиййэщэййхэщйшубпнщййшпййопцкп

In [123]:
key1 = "не  ошибается  тот  кто ничего  не  делает"

In [118]:
"-+".join(["aa", "bb", "cc"])

'aa-+bb-+cc'

In [ ]:
abcde
b
1
1-2
-1


In [116]:
"abcde"[-1]

'e'

In [ ]:
key1 = caesar_brootforce(chars)

In [ ]:
после этого вы обратили свой взор на второе изображение. слишком оно обычное и не привлекает внимание. 
возможно в нем спрятано сообщение
загрузив его вы решили посмотреть на четности чисел каждого пикселя и увидели что они совпадают всегда во всех трех каналах.
а это означает что надо вытащить четности каждого пикселя как бинарный код

In [ ]:
filepath = "222.bmp"
with Image.open(filepath) as img:
    img.load()
    img = np.asarray(img)
new_img = deepcopy(img)

In [ ]:
def get_bin_array(img):
    pass

bin_array = get_bin_array(new_img)

Далее вы разбили двоичный код по 8 бит и каждый преобразовали в десятичное число а затем и в букву аналогично тому как сделали это раньше (32 пробел к остальным надо добавить 1000 и взять chr)

In [ ]:
def get_char_from_bin_array(bin_array):
    pass


char_array = get_char_from_bin_array(bin_array)

сообщение оказалось длинным и зашифрованным. но у вас есть ключ. наудачу вы попробовали расшифровать код шифром Вижинера

In [ ]:
def vigenere_encryptor(text: str, key:str) -> str:
    """
    Зашифровать с помощью шифра Вижинера. 
    Шифр Вижинера это продвинутый шифр Цезаря в котором ключ для шифра цезаря меняется каждую следующую букву.
    Но меняется не случайно, а в соответствии с ключем-фразой для шифра Вижинера. Ключ-фраза определяет какой следующий
    будет ключ для шифра Цезаря: берем следующую букву ключа Вижинера ищем ее порядок в алфавите 
    "абвгдеёжзийклмнопрстуфхцчшщъыьэюя " и этот индекс и есть 
    смещение-ключ для шифра Цезаря.
    """
    pass


def vigenere_decryptor(text: str, key:str) -> str:
    """
    """
    pass

In [ ]:
text = vigenere_decryptor(char_array, key1)

In [4]:
import string

In [2]:
s = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "
len(s)

35